# 🗂️ Crear un Internal Load Balancer

En este laboratorio, configurarás un **Internal Load Balancer (ILB)** en Google Cloud para balancear tráfico TCP/UDP dentro de una red privada (**VPC**). Este enfoque permite ejecutar y escalar servicios internos, como sistemas contables, accesibles solo para instancias internas, garantizando seguridad y eficiencia. Crearás dos **Managed Instance Groups (MIG)** en la misma región y probarás el ILB con una máquina utilitaria.

💡 **Beneficio empresarial**: Un **ILB** asegura que los servicios internos, como los sistemas de contabilidad, estén protegidos dentro de la red privada, optimizando la disponibilidad y reduciendo costos al distribuir eficientemente la carga sin exponer datos sensibles al exterior.

Para más información, consulta la [Documentación de Google Cloud Load Balancing](https://cloud.google.com/load-balancing/docs) y la [Documentación de Google Cloud VPC](https://cloud.google.com/vpc/docs).

## 🎯 Objetivos
- 🔐 Crear reglas de firewall para HTTP y **health checks**.
- 🛠️ Configurar dos **instance templates**.
- 🖥️ Crear dos **managed instance groups (MIG)**.
- ⚖️ Configurar y probar un **Internal Load Balancer**.

📊 **Concepto clave**: Un **Internal Load Balancer** distribuye tráfico dentro de una red privada, aislando servicios sensibles y mejorando la disponibilidad sin necesidad de IPs públicas.

## 🏗️ Tarea 1: Configurar reglas de firewall HTTP y de health check

En esta tarea, crearás reglas de firewall para permitir tráfico HTTP y **health checks** en la red precreada `myinternalapp`.

### Paso 1.1: Explorar la red myinternalapp

1. Ve a **Navigation menu** > **VPC network** > **VPC networks**.
2. Revisa la red `myinternalapp`, que incluye:
   - Subredes: `subneta` y `subnetb`.
   - Reglas de firewall predefinidas para RDP, SSH e ICMP.

**Explicación**:
- La red `myinternalapp` actúa como un entorno privado para servicios internos, como sistemas contables, con subredes segmentadas para mayor organización.

💡 **Contexto empresarial**: Esta red es como un departamento contable virtual que opera de forma aislada, con reglas de acceso predefinidas para proteger datos financieros.

### Paso 1.2: Crear regla HTTP (app-allow-http)

1. Ve a **VPC network** > **Firewall rules** > **+ Create Firewall Rule**.
2. Configura:
   - **Name**: `app-allow-http`
   - **Network**: `my-internal-app`
   - **Targets**: **Specified target tags** → `lb-backend`
   - **Source IPv4 ranges**: `10.10.0.0/16`
   - **Protocols/ports**: `TCP 80`
3. Haz clic en **Create**.

**Explicación**:
- Esta regla permite tráfico HTTP (puerto 80) desde la red interna (`10.10.0.0/16`) hacia instancias con la etiqueta `lb-backend`.

💡 **Contexto empresarial**: Esto es como permitir que solo los empleados de un departamento contable accedan a un sistema financiero interno.

### Paso 1.3: Crear regla de health check (app-allow-health-check)

1. Haz clic en **+ Create Firewall Rule**.
2. Configura:
   - **Name**: `app-allow-health-check`
   - **Network**: `my-internal-app`
   - **Targets**: **Specified target tags** → `lb-backend`
   - **Source IPv4 ranges**: `130.211.0.0/22 35.191.0.0/16` (separadas por espacio)
   - **Protocols/ports**: `TCP` (todos los puertos)
3. Haz clic en **Create**.

**Explicación**:
- Esta regla permite que los **health checks** de Google Cloud (desde los rangos de IP especificados) verifiquen la salud de las instancias con la etiqueta `lb-backend`.

💡 **Contexto empresarial**: Los **health checks** son como auditorías automáticas que confirman que un sistema contable está operativo, asegurando continuidad del servicio.

## 🏗️ Tarea 2: Configurar instance templates y crear instance groups

En esta tarea, crearás plantillas de instancias (**instance templates**) y grupos de instancias gestionados (**MIG**) para los backends del ILB.

### Paso 2.1: Instance template 1 (subneta)

1. Ve a **Compute Engine** > **Instance templates** > **Create instance template**.
2. Configura:
   - **Name**: `instance-template-1`
   - **Location**: `Global`
   - **Series**: `E2`
   - **Machine type**: `e2-micro`
   - **Network tags**: `lb-backend`
   - **Network/Subnetwork**: `myinternalapp` / `subneta`
   - **External IPv4**: `None`
   - **Metadata**:
     - **Key**: `startup-script-url`
     - **Value**: `gs://cloud-training/gcpnet/ilb/startup.sh`
3. Haz clic en **Create**.

**Explicación**:
- El **startup script** instala **Apache** y personaliza la página de bienvenida con la IP del cliente, nombre, región y zona de la VM.
- La ausencia de IP externa asegura que las VMs solo sean accesibles dentro de la red privada.

💡 **Contexto empresarial**: El **startup script** es como un procedimiento automatizado que configura un sistema contable con información específica al iniciarse, garantizando consistencia.

### Paso 2.2: Instance template 2 (subnetb)

1. Selecciona `instance-template-1` > **+ CREATE SIMILAR**.
2. Cambia:
   - **Name**: `instance-template-2`
   - **Subnetwork**: `subnetb`
3. Mantén el tag `lb-backend` y haz clic en **Create**.

**Explicación**:
- Esto crea una plantilla idéntica para una subred diferente, asegurando que las instancias estén distribuidas entre subredes para mayor resiliencia.

### Paso 2.3: Crear Managed Instance Group 1

1. Ve a **Compute Engine** > **Instance groups** > **Create instance group**.
2. Configura:
   - **Name**: `instance-group-1`
   - **Template**: `instance-template-1`
   - **Location**: **Single zone** → Región `REGION`, Zona `ZONE` (por ejemplo, `us-central1`, `us-central1-a`)
   - **Autoscaling**: Min `1`, Max `1`, señal `CPU 80%`, período `45 s`
3. Haz clic en **Create**.

### Paso 2.4: Crear Managed Instance Group 2

Repite el proceso con:
- **Name**: `instance-group-2`
- **Template**: `instance-template-2`
- **Location**: Otra zona en la misma región (por ejemplo, `us-central1-b`).

**Explicación**:
- Los **MIG** garantizan uniformidad y permiten escalado automático, aunque en este caso están configurados con una sola instancia por grupo para simplicidad.

💡 **Contexto empresarial**: Un **MIG** es como un equipo contable automatizado que se mantiene consistente y puede escalar durante picos de trabajo, como cierres fiscales.

### Paso 2.5: Verificar backends

1. Ve a **Compute Engine** > **VM instances**.
2. Verifica que existan dos VMs con IPs internas:
   - `instance-group-1`: `10.10.20.2`
   - `instance-group-2`: `10.10.30.2`
3. Crea una VM utilitaria:
   - **Name**: `utility-vm`
   - **Region/Zone**: `REGION` / `ZONE` (por ejemplo, `us-central1` / `us-central1-a`)
   - **Type**: `e2-micro`
   - **Subnetwork**: `subneta`
   - **Internal IP**: `10.10.20.50`

**Explicación**:
- La VM `utility-vm` se usará para probar el ILB desde dentro de la red privada.

💡 **Contexto empresarial**: La VM utilitaria es como una estación de auditoría interna que verifica el acceso a los sistemas financieros.

#### Comandos de verificación desde utility-vm

1. Abre una sesión **SSH** en `utility-vm`.
2. Ejecuta:

In [ ]:
curl 10.10.20.2

**Explicación**:
- `curl`: Realiza una solicitud HTTP al puerto 80.
- `10.10.20.2`: IP interna de la VM en `instance-group-1`.
- La respuesta muestra la página de bienvenida generada por **Apache**, incluyendo la IP del cliente, nombre, región y zona de la VM.

In [ ]:
curl 10.10.30.2

**Explicación**:
- Consulta la VM en `instance-group-2` (`10.10.30.2`), mostrando información similar.

In [ ]:
exit

**Explicación**:
- Cierra la sesión SSH.

**Campos que identifican la ubicación del backend**:
- **Server Hostname**: Nombre de la VM.
- **Server Location**: Región y zona de la VM.
- **Client IP**: IP de la VM que realiza la petición (`utility-vm`).

💡 **Contexto empresarial**: Estos comandos son como verificar que los registros financieros de diferentes sucursales son accesibles desde una estación de auditoría interna.

## 🏗️ Tarea 3: Configurar el Internal Load Balancer

En esta tarea, configurarás un **Internal Load Balancer** para distribuir tráfico entre los dos **MIG**.

### Paso 3.1: Inicio de configuración

1. Ve a **Navigation menu** > **View All Products** > **Network Services** > **Load balancing** > **Create load balancer**.
2. Selecciona:
   - **Type**: `Network Load Balancer (TCP/UDP/SSL)`
   - **Proxy/passthrough**: `Passthrough`
   - **Exposition**: `Internal`
3. Haz clic en **CONFIGURE**.
4. Configura:
   - **Name**: `my-ilb`
   - **Region**: `REGION` (por ejemplo, `us-central1`)
   - **Network**: `my-internal-app`

### Paso 3.2: Backend service

1. Selecciona **Backend configuration**.
2. Añade backends:
   - **Instance group**: `instance-group-1`
   - **Add a backend** → `instance-group-2`
3. Crea un **health check**:
   - **Name**: `my-ilb-health-check`
   - **Protocol**: `TCP`
   - **Port**: `80`
   - **Check interval**: `5 s`
   - **Healthy threshold**: `2 successes`
   - **Unhealthy threshold**: `2 failures`
4. Verifica que aparezca un ✔️ azul junto a **Backend configuration**.

### Paso 3.3: Frontend

1. Selecciona **Frontend configuration**.
2. Configura:
   - **Subnetwork**: `subnetb`
   - **Internal IP** → **Create IP address**:
     - **Name**: `my-ilb-ip`
     - **Static IP address**: `Let me choose`
     - **Custom IP**: `10.10.30.5`
     - **Port number**: `80`
3. Haz clic en **Done**.

### Paso 3.4: Revisión y creación

1. Haz clic en **Review and finalize**.
2. Verifica las configuraciones de **Backend** y **Frontend**.
3. Haz clic en **Create** y espera a que el ILB esté aprovisionado.

**Explicación**:
- El ILB distribuye tráfico interno a los **MIG** según el estado de los **health checks** y el algoritmo de balanceo.

💡 **Contexto empresarial**: El ILB es como un sistema interno que distribuye solicitudes financieras entre diferentes servidores contables, asegurando alta disponibilidad sin exponer datos al exterior.

## 🧪 Tarea 4: Probar el Internal Load Balancer

En esta tarea, probarás el ILB desde `utility-vm` para verificar que distribuye tráfico entre los **MIG**.

1. Abre una sesión **SSH** en `utility-vm`.
2. Ejecuta:

In [ ]:
curl 10.10.30.5

**Explicación**:
- `curl`: Envía una solicitud HTTP al puerto 80.
- `10.10.30.5`: IP interna estática del frontend del ILB.
- El ILB distribuye la solicitud a uno de los **MIG** (`instance-group-1` o `instance-group-2`), mostrando la página personalizada.

3. Repite el comando varias veces. Deberías alternar entre respuestas de `instance-group-1` (zona A) y `instance-group-2` (zona B), confirmando el balanceo.

💡 **Contexto empresarial**: Esto verifica que el sistema contable interno distribuye solicitudes correctamente entre diferentes servidores, asegurando continuidad y eficiencia.

## 📚 Conceptos teóricos clave

| **Término**             | **Explicación para negocios**                                                                 |
|-------------------------|---------------------------------------------------------------------------------------------|
| **Internal Load Balancer (ILB)** | Un “distribuidor de tráfico” privado dentro de la red. Aísla servicios sensibles, mejora disponibilidad y reparte carga ➜ menos riesgo de caída y mejor uso de recursos. |
| **Managed Instance Group (MIG)** | Conjunto automático de VMs idénticas. Simplifica escalamiento (↑/↓ instancias) controlando costos y manteniendo rendimiento. |
| **Health Check**        | “Pulso de salud” que vigila cada VM. Si falla, el ILB deja de enviarle tráfico —garantizando continuidad del servicio. |
| **Firewall Rules**      | Guardias de acceso. Permiten o bloquean tráfico según IP, puerto y protocolo. Mantienen la seguridad y reducen superficie de ataque. |
| **Autoscaling por CPU** | Ajusta el número de VMs según uso real del procesador, optimizando gastos operativos. |

## 🚀 Resumen

💡 **Conclusión empresarial**: Configurar un **Internal Load Balancer** permite a las empresas mantener servicios internos, como sistemas contables, seguros y disponibles dentro de una red privada. Los **MIG** y **health checks** garantizan escalabilidad y continuidad, mientras que las reglas de firewall protegen contra accesos no autorizados, optimizando costos y cumpliendo con normativas de seguridad.

Para más información, consulta la [Documentación de Google Cloud Load Balancing](https://cloud.google.com/load-balancing/docs) y la [Documentación de Google Cloud VPC](https://cloud.google.com/vpc/docs).

**Nota sobre imágenes**: El documento menciona `image1.png` y `image2.png`, probablemente diagramas de red. Si necesitas que se analicen o incorporen, proporciona las imágenes o más detalles.